> ************************
>
>  有个新的想法
>
>  在cola算法中加上一个步骤：
>
>  可以修改precision and recall
>
> ************************

## Test:

In [24]:
import dataset
from xai_cola import data,ml_model,counterfactual_explainer,cola,counterfactual_refiner
from xai_cola.cola_explainer import COunterfactualwithLimitedActions

[autoreload of xai_cola.ml_model.model failed: Traceback (most recent call last):
  File "c:\Users\ZhuLi\anaconda3\envs\COLA\lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\ZhuLi\anaconda3\envs\COLA\lib\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "c:\Users\ZhuLi\anaconda3\envs\COLA\lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\ZhuLi\anaconda3\envs\COLA\lib\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "c:\Users\ZhuLi\anaconda3\envs\COLA\lib\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "c:\Users\ZhuLi\anaconda3\envs\COLA\lib\site-packages\IPython\extensions\autoreload.py", line 309, in update_function
    setattr(old, name, getattr

In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
dataset1 =dataset.GermanCreditDataset()
df1 = dataset1.get_dataframe()
df1.head(2)

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1


In [27]:
# dataset2 = dataset.HotelBookingsDataset()
# df2 = dataset2.get_dataframe()
# df2.head(2)

### Configuration

In [28]:
data_without_target = df1.drop(columns=['Risk']).copy()
data_without_target = data_without_target.head(5)


In [29]:
feature_names = data_without_target.columns
feature_names

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose'],
      dtype='object')

In [30]:
data1 = data.PandasData(data=df1, target_name="Risk")

In [31]:
data2 = data.PandasData(data=data_without_target, target_name="Risk")

In [32]:
import joblib
import pickle

In [33]:
lgbmcClassifier = joblib.load('LGBMClassifier.pkl')
print(f'----LGBMClassifier.pkl model has been loaded----')

----LGBMClassifier.pkl model has been loaded----


In [11]:
scaled_lgbmcClassifier = joblib.load('scaled_LGBMClassifier.pkl')
print(f'---scaledLGBMClassifier.pkl has already been loaded---')

---scaledLGBMClassifier.pkl has already been loaded---


In [12]:
scaled_PyTorchDNN = joblib.load('scaled_PyTorchDNN.pkl')
print(f'---scaled_PyTorchDNN.pkl has already been loaded---')

---scaled_PyTorchDNN.pkl has already been loaded---


In [13]:
scaled_PyTorchDNN.classifier.__class__.__name__ 

'PyTorchDNN'

In [14]:
# 从文件加载Scaler
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)
print(f'--- scaler has already been loaded ---')

--- scaler has already been loaded ---


In [15]:
scaled_data = loaded_scaler.fit_transform(data_without_target)

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [16]:
scaled_data.shape

(1000, 9)

In [17]:
scaled_data

array([[ 2.76645648,  0.67028006,  0.14694918, ..., -0.74513141,
        -1.23647786,  1.07326286],
       [-1.19140394, -1.49191369,  0.14694918, ...,  0.94981679,
         2.24819436,  1.07326286],
       [ 1.18331231,  0.67028006, -1.38377145, ..., -0.41656241,
        -0.73866754,  0.06170503],
       ...,
       [ 0.21583532,  0.67028006,  0.14694918, ..., -0.87450324,
        -0.73866754,  1.07326286],
       [-1.10345149,  0.67028006,  0.14694918, ..., -0.50552769,
         1.9992892 ,  1.07326286],
       [-0.75164167,  0.67028006,  0.14694918, ...,  0.46245715,
         1.9992892 , -0.9498528 ]])

In [18]:
data_alibi = data.NumpyData(
    data=scaled_data, 
    target_name='Risk', 
    feature_names=feature_names
    )

In [19]:
data_alibi.get_dataframe()

In [41]:
ml_model1 = ml_model.Model(model=lgbmcClassifier, backend='sklearn')  

In [21]:
scaled_ml_model = ml_model.Model(model = scaled_lgbmcClassifier, backend = 'sklearn')

In [22]:
scaled_PyT_DNN = ml_model.Model(model = scaled_PyTorchDNN, backend= 'PYT')

In [23]:
scaled_PyT_DNN.model.classifier.__class__.__name__

'PyTorchDNN'

### DiCE

In [42]:
counterfactual_explainer1 = counterfactual_explainer.DiCE(ml_model=ml_model1)  #will add more input params according to the dice_ml algorithm 

In [43]:
a,b = counterfactual_explainer1.generate_counterfactuals(data=data2)

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  0%|          | 0/5 [00:00<?, ?it/s]np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
 20%|██        | 1/5 [00:00<00:00,  8.63it/s]np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.o

In [16]:
a

array([[  67,    1,    2,    1,    0,    1, 1169,    6,    5],
       [  22,    0,    2,    1,    1,    2, 5951,   48,    5],
       [  49,    1,    1,    1,    1,    0, 2096,   12,    3],
       [  45,    1,    2,    0,    1,    1, 7882,   42,    4],
       [  53,    1,    2,    0,    1,    1, 4870,   24,    1]],
      dtype=int64)

In [17]:
b

array([[  22,    0,    2,    1,    1,    2, 5951,   20,    2],
       [  39,    1,    1,    0,    1,    1, 4870,   24,    1],
       [  49,    1,    1,    1,    1,    0, 6537,   12,    3],
       [  45,    1,    2,    0,    1,    0, 7013,   42,    4],
       [  67,    1,    2,    0,    0,    1, 5319,    6,    5]],
      dtype=int64)

### KNN - 1vn counterfactual explanation

In [15]:
from xai_cola.counterfactual_explainer import KNN

knn = KNN(ml_model=ml_model1)

In [16]:
c,d = knn.generate_counterfactuals(data=data2, n_neighbors=2)

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
X does not have valid feature names, but EfficientQuantileTransformer was fitted with feature names
X does not have valid feature names, but EfficientQuantileTransformer was fitted with feature names


estimate:[array([[5.300e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        4.870e+03, 2.400e+01, 1.000e+00],
       [2.200e+01, 0.000e+00, 2.000e+00, 1.000e+00, 1.000e+00, 2.000e+00,
        5.951e+03, 4.800e+01, 5.000e+00]]), array([[4.500e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        7.882e+03, 4.200e+01, 4.000e+00],
       [6.700e+01, 1.000e+00, 2.000e+00, 1.000e+00, 0.000e+00, 1.000e+00,
        1.169e+03, 6.000e+00, 5.000e+00]]), array([[5.300e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        4.870e+03, 2.400e+01, 1.000e+00],
       [2.200e+01, 0.000e+00, 2.000e+00, 1.000e+00, 1.000e+00, 2.000e+00,
        5.951e+03, 4.800e+01, 5.000e+00]]), array([[5.300e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        4.870e+03, 2.400e+01, 1.000e+00],
       [2.200e+01, 0.000e+00, 2.000e+00, 1.000e+00, 1.000e+00, 2.000e+00,
        5.951e+03, 4.800e+01, 5.000e+00]]), array([[4.500e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.

X does not have valid feature names, but EfficientQuantileTransformer was fitted with feature names
X does not have valid feature names, but EfficientQuantileTransformer was fitted with feature names
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [17]:
data2.get_x().columns

Index(['Age', 'Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Credit amount', 'Duration', 'Purpose'],
      dtype='object')

In [22]:
ml_model1.predict(c)

array([1, 0, 0, 0, 1], dtype=int64)

In [18]:
c

array([[  53,    1,    2,    0,    1,    1, 4870,   24,    1],
       [  45,    1,    2,    0,    1,    1, 7882,   42,    4],
       [  67,    1,    2,    1,    0,    1, 1169,    6,    5],
       [  49,    1,    1,    1,    1,    0, 2096,   12,    3],
       [  22,    0,    2,    1,    1,    2, 5951,   48,    5]],
      dtype=int64)

In [23]:
ml_model1.predict(d)

array([0, 1, 1, 1, 0], dtype=int64)

In [20]:
d

array([[4.500e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        7.882e+03, 4.200e+01, 4.000e+00],
       [5.300e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        4.870e+03, 2.400e+01, 1.000e+00],
       [5.300e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        4.870e+03, 2.400e+01, 1.000e+00],
       [5.300e+01, 1.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
        4.870e+03, 2.400e+01, 1.000e+00],
       [6.700e+01, 1.000e+00, 2.000e+00, 1.000e+00, 0.000e+00, 1.000e+00,
        1.169e+03, 6.000e+00, 5.000e+00]])

In [76]:
d.shape

(5, 9)

### ALIBI - 1v1 counterfactual explanation

In [30]:
from xai_cola.counterfactual_explainer import AlibiCounterfactualInstances
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


In [31]:
data_alibi.get_dataframe()

In [32]:
alibi = AlibiCounterfactualInstances(
    ml_model=scaled_ml_model
    )

Predicted class for factual instance 0: [1]
Target class for counterfactual instance 0: 0




The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


Predicted class for factual instance 1: [1]
Target class for counterfactual instance 1: 0


The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


Predicted class for factual instance 2: [1]
Target class for counterfactual instance 2: 0


The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


In [33]:
print(f'alibi.factual: {alibi.factual}')
print(f'alibi.counterfactual: {alibi.counterfactual}')

alibi.factual: [[-4.87784309e-01 -1.49191369e+00  1.46949180e-01 -1.33710455e-01
  -1.96609009e-01  1.04437190e+00 -8.34096700e-01 -9.87572702e-01
   5.67483943e-01]
 [-1.36730885e+00 -1.49191369e+00  1.46949180e-01  1.74953540e+00
  -1.96609009e-01 -1.04541732e-03 -4.36765680e-01 -2.40857225e-01
   5.67483943e-01]
 [-5.75736763e-01 -1.49191369e+00 -1.38377145e+00  1.74953540e+00
  -1.96609009e-01 -1.04541732e-03 -5.71454161e-01  2.56953092e-01
   1.07326286e+00]]
alibi.counterfactual: [[-0.4877843  -1.4919137   0.14694919 -0.13371046 -0.196609    1.0443718
  -0.84165835 -0.9875727   0.56748396]
 [-1.3673089  -1.4919137   0.14694919  1.7495354  -0.196609    0.02990204
  -0.43893468 -0.24085723  0.56748396]
 [-0.57573676 -1.4919137  -1.3837714   1.7495354  -0.196609   -0.00688358
  -0.5861846   0.2569531   1.0732628 ]]


In [34]:
import numpy as np
import pandas as pd
import alibi
x_factual_pandas = data1.get_x()
target_name = data1.get_target_name()
sample_num = 4

def get_factual_indices():
    x_factual_ext = x_factual_pandas.copy()
    predict = scaled_ml_model.predict(x_factual_pandas)
    x_factual_ext[target_name] = predict
    sampling_weights = np.exp(x_factual_ext[target_name].values.clip(min=0) * 4)
    indices = (x_factual_ext.sample(sample_num, weights=sampling_weights)).index
    return indices, x_factual_ext


In [35]:
# def generate_counterfactuals(self) -> pd.DataFrame:

indices, _ = get_factual_indices()
x_chosen = x_factual_pandas.loc[indices]

# 对每个实例生成反事实
counterfactuals = []

In [36]:
# x_factual = x_chosen[i].reshape(1, -1)  # 取出第 i 行数据并重塑为 (1, n) 形状
x_factual = x_chosen.iloc[[0]]
print(x_chosen)
print("-------------------")
print(x_factual)
print("-------------------")
predicted_class = scaled_ml_model.predict(x_factual)
print(predicted_class)
print("-------------------")
# print(f'Predicted class for factual instance {0}: {predicted_class}')

     Age  Sex  Job  Housing  Saving accounts  Checking account  Credit amount  \
996   40    1    3        1                1                 1           3857   
430   74    1    1        1                1                 0           3448   
629   64    1    1        1                0                 0           3832   
804   24    0    0        2                0                 2           7472   

     Duration  Purpose  
996        30        1  
430         5        0  
629         9        3  
804        12        1  
-------------------
     Age  Sex  Job  Housing  Saving accounts  Checking account  Credit amount  \
996   40    1    3        1                1                 1           3857   

     Duration  Purpose  
996        30        1  
-------------------
[1]
-------------------


In [37]:
# 设置目标类，反事实的目标是与原始预测结果相反
target_class = 1 if predicted_class == 0 else 0
print(f'Target class for counterfactual instance {0}: {target_class}')

Target class for counterfactual instance 0: 0


In [38]:
(1,) + x_factual.shape[1:]

(1, 9)

In [39]:
# 定义反事实解释器
# cf = alibi.explainers.Counterfactual(
#     predict_fn=ml_model1.predict_proba, 
#     shape=x_factual.shape, 
#     distance_fn='l1', 
#     target_proba=1.0,
#     target_class='other', 
#     max_iter=1000, early_stop=50, lam_init=1e-1,
#     max_lam_steps=10, tol=0.05, learning_rate_init=0.1,
#     feature_range=(-1e10, 1e10), eps=0.01, init='identity',
#     decay=True, write_dir=None, debug=False)
explainer =alibi.explainers.Counterfactual(
    scaled_ml_model.predict_proba,                              # The model to explain
    shape=(1,) + x_factual.shape[1:],     # The shape of the model input
    target_proba=0.6,                  # The target class probability
    tol=0.7,                         # The tolerance for the loss
    # target_class='other',               # The target class to obtain  
    target_class = 0,
    max_iter=5000,
    lam_init=1e-4,
    max_lam_steps=40,  # 增加 lambda 调整步数
)


In [40]:
scaled_x_factual = loaded_scaler.transform(x_factual)

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [41]:
# 生成反事实实例
explanation = explainer.explain(scaled_x_factual)
# counterfactuals.append(explanation.cf)

In [42]:
scaled_x_factual

array([[ 3.91740230e-01,  6.70280063e-01,  1.67766981e+00,
        -1.33710455e-01, -1.96609009e-01, -1.04541732e-03,
         2.07612369e-01,  7.54763410e-01, -9.49852804e-01]])

In [43]:
explanation['cf']

The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


{'X': array([[ 0.39174023,  0.67028004,  1.6776698 , -0.13371046, -0.196609  ,
         -0.01986519,  0.23911563,  0.7547634 , -0.9498528 ]],
       dtype=float32),
 'distance': 0.050323035567998886,
 'lambda': 9.999999997904524e-05,
 'index': 155010,
 'class': 0,
 'proba': array([[0.74901532, 0.25098468]]),
 'loss': 0.02221059725758396}

In [44]:
x_counterfactual = loaded_scaler.inverse_transform(explanation['cf']['X'])
x_counterfactual

The Explanation object is not a dictionary anymore and accessing elements should be done via attribute access. Accessing via item will stop working in a future version.


array([[4.0000000e+01, 1.0000000e+00, 3.0000000e+00, 1.0000000e+00,
        1.0000000e+00, 9.8199785e-01, 3.9458809e+03, 3.0000000e+01,
        1.0000000e+00]], dtype=float32)

In [45]:
x_factual

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
996,40,1,3,1,1,1,3857,30,1


In [46]:
ml_model1.predict(x_factual)

array([0], dtype=int64)

In [47]:
ml_model1.predict(x_counterfactual)

array([1], dtype=int64)

#### DisCount - only compatiable with PYT

In [48]:
# counterfactual_explainer_discount = counterfactual_explainer.DisCount(
#     ml_model = scaled_PyT_DNN,
#     data = data1,
#     sample_num = 4,
# )

In [49]:
### DisCount is not compatiable with scikit-learn
# counterfactual_explainer2 = counterfactual_explainer.DisCount(ml_model=ml_model1, data=data1, sample_num=4)

### *choose policy and make limitation


In [18]:
ot_pshap_max = COunterfactualwithLimitedActions(
    data=data2, 
    ml_model=ml_model1, 
    x_factual=a,
    x_counterfactual=b,
    limited_actions=3, 
    matcher="ot", # joint_probability
    attributor="pshap", # used to get the varphi
    Avalues_method="max" # get the q
    )

In [56]:
# ot_pshap_max = COunterfactualwithLimitedActions(
#     data=data1, 
#     ml_model=ml_model1, 
#     x_factual=x_factual,
#     x_counterfactual=x_counterfactual,
#     limited_actions=10, 
#     matcher="ot", # joint_probability
#     attributor="pshap", # used to get the varphi
#     Avalues_method="max" # get the q
#     )

In [19]:
factual, ce, ace = ot_pshap_max.generate_results()

INFO:shap:num_full_subsets = 3
INFO:shap:phi = [-0.08262096  0.03448063  0.01142397 -0.11477744  0.16859833 -0.25255664
 -0.07098289]
INFO:shap:num_full_subsets = 3
INFO:shap:phi = [ 0.18470638  0.03342442  0.03062819  0.23597029 -0.00842148  0.52278706
 -0.08008822]
INFO:shap:num_full_subsets = 4
INFO:shap:phi = [ 0.01329149 -0.0262632  -0.00825354  0.00401998 -0.06431161 -0.03795905
  0.06636484  0.02551573]
INFO:shap:num_full_subsets = 1
INFO:shap:phi = [0.22210661 0.03589675]
INFO:shap:num_full_subsets = 3
INFO:shap:phi = [ 0.29109896 -0.0355451  -0.04918497  0.07103084  0.38973948  0.05048516
 -0.00780808]


In [20]:
factual

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,48,5,1
2,49,1,1,1,1,0,2096,12,3,0
3,45,1,2,0,1,1,7882,42,4,0
4,53,1,2,0,1,1,4870,24,1,1


In [21]:
ce

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,22,0,2,1,1,2,5951,20,2,0
1,39,1,1,0,1,1,4870,24,1,0
2,49,1,1,1,1,0,6537,12,3,0
3,45,1,2,0,1,0,7013,42,4,0
4,67,1,2,0,0,1,5319,6,5,0


In [22]:
ace

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,1,2,1,0,1,1169,6,5,0
1,22,0,2,1,1,2,5951,12,5,0
2,49,1,1,1,1,0,2096,12,3,0
3,45,1,2,0,1,1,7882,42,4,0
4,22,1,2,0,1,1,5951,24,1,0


### lime explainer - feature attribution

In [61]:
probabilities = ml_model1.predict_proba(x_factual)
print("Shape of probabilities:", probabilities.shape)
print("Probabilities:", probabilities)

Shape of probabilities: (4, 2)
Probabilities: [[0.34389295 0.65610705]
 [0.07014478 0.92985522]
 [0.04131984 0.95868016]
 [0.33687725 0.66312275]]


In [62]:
import numpy as np
import pandas as pd
from lime.lime_tabular import LimeTabularExplainer

In [63]:
sample_num = 4

def get_factual_indices():
    x_factual_ext = data1.get_x().copy()
    predict = ml_model1.predict(x_factual_ext)
    x_factual_ext[data1.get_target_name()] = predict
    sampling_weights = np.exp(x_factual_ext[data1.get_target_name()].values.clip(min=0) * 4)
    indices = (x_factual_ext.sample(sample_num, weights=sampling_weights)).index
    return indices, x_factual_ext


In [64]:
indices, x_factual_ext = get_factual_indices()
x_chosen = data1.get_x().loc[indices] # dataframe without 'Risk'/ target
x_chosen

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
598,31,1,2,1,1,0,2775,18,1
197,27,0,2,2,2,2,951,12,4
259,57,0,1,1,2,0,1154,11,5
54,57,1,2,0,1,2,2225,36,1


In [65]:
x_chosen.iloc[0]

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


Age                   31
Sex                    1
Job                    2
Housing                1
Saving accounts        1
Checking account       0
Credit amount       2775
Duration              18
Purpose                1
Name: 598, dtype: int64

In [66]:
# 初始化LIME解释器
explainer = LimeTabularExplainer(
    training_data=data1.get_x().values,  #  training data should be numpy
    feature_names=data1.get_x_labels(), 
    class_names=['good credit', 'bad credit'], 
    discretize_continuous=False)


np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [67]:
# 选择4个样本并生成反事实解释
sample_data = x_chosen.values

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [68]:
for i in range(len(sample_data)):
    # LIME的解释对象
    exp = explainer.explain_instance(sample_data[i], 
                                     ml_model1.predict_proba, 
                                     num_features=len(data1.get_x_labels()))

    # 打印原始预测
    original_prediction = ml_model1.predict([sample_data[i]])[0]
    print(f"Original Prediction for sample {i+1}: {original_prediction}")

    # 打印反事实解释
    print(f"Counterfactual explanation for sample {i+1}:")
    for feature, weight in exp.as_list():
        print(f" - {feature}: {weight}")
    print("\n")


Original Prediction for sample 1: 1
Counterfactual explanation for sample 1:
 - Duration: 0.11302070689821693
 - Credit amount: -0.056911312818798264
 - Checking account: 0.04429573221082862
 - Housing: 0.01910032827385068
 - Job: -0.009198971404145202
 - Age: -0.007097459635038736
 - Sex: -0.006889817077267384
 - Purpose: -0.0023307998469790777
 - Saving accounts: -0.001630207871041489


Original Prediction for sample 2: 1
Counterfactual explanation for sample 2:
 - Duration: 0.1180417717391339
 - Credit amount: -0.05415172792897047
 - Housing: 0.02283568367425364
 - Checking account: 0.01636318503351073
 - Job: -0.008076677306929868
 - Saving accounts: -0.007846844405687975
 - Sex: -0.006031471791645677
 - Purpose: -0.00558727568676809
 - Age: -0.004802725575955554


Original Prediction for sample 3: 0
Counterfactual explanation for sample 3:
 - Duration: 0.1269047226231337
 - Checking account: 0.055410414900948196
 - Credit amount: -0.039249687109373
 - Purpose: -0.02462717393869595